In [1]:
from pathlib import Path
from loguru import logger
from typing import Any
import pandas as pd

In [2]:
signatures_path = Path("../data/sigs.csv")
df = pd.read_csv(signatures_path)
logger.info("loaded signatures data: {} rows {} columns", df.shape[0], df.shape[1])
old_name = df.columns[0]
df.rename(columns={old_name:"cell_id"}, inplace=True)

2024-06-16 20:39:42.509 | INFO     | __main__:<module>:3 - loaded signatures data: 9370 rows 3 columns


In [3]:

train_df = df.sample(frac=0.1)
logger.info("loaded signatures training data: {} rows {} columns", train_df.shape[0], train_df.shape[1])
test_df = df.drop(train_df.index)
logger.info("loaded signatures data: {} rows {} columns", test_df.shape[0], test_df.shape[1])

2024-06-16 20:39:44.000 | INFO     | __main__:<module>:2 - loaded signatures training data: 937 rows 3 columns
2024-06-16 20:39:44.005 | INFO     | __main__:<module>:4 - loaded signatures data: 8433 rows 3 columns


In [4]:
cluster_table: dict[int, list[str]] = {}
for cluster in df.cluster.unique():
    # cluster_table[cluster] = []
    cluster_df = train_df[df.cluster==cluster]
    cluster_table[cluster] = cluster_df.signature
assert len(cluster_table)

len(cluster_table[0])

/tmp/ipykernel_2010582/2082437146.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cluster_df = train_df[df.cluster==cluster]


127

In [5]:
import openai
import dotenv
import os
from openai import OpenAI 

dotenv.load_dotenv("../.env")

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

def get_embeddings(texts):
    client = openai.OpenAI()  # Create an OpenAI client
    model = "text-embedding-ada-002"  # Use the latest embedding model
    response = client.embeddings.create(
        input=texts,
        model=model
    )
    # embeddings = [data['embedding'] for data in response['data']]
    return response.model_dump()['data'] # type: ignore 

# texts = cluster_table[0]
# logger.info("processing {} inputs")
# embeddings = get_embeddings(texts)
# logger.info("obtained {} embeddings", len(embeddings))

In [75]:
# embeds = [e['embedding'] for e in embeddings]
# logger.info("extracted {} embeddings", len(embeds))

2024-06-14 18:59:48.463 | INFO     | __main__:<module>:2 - extracted 115 embeddings


In [6]:
train_df.head()

,cell_id,cluster,signature
3989,GTGAGGAGTGGATTAT,4,TGFBR2 FAM172A CCSER2 CLEC2D DOCK11 AC079793.1...
6736,ATCACTAAGGCATGTT,8,PDE4D RALGPS2 ZBTB11 LINC01505 SSH2 TCF4 CBLB ...
599,ACCCAACCATAATCCG,0,PPP1R12A PTPRJ SKAP1 PIK3CD CDC14A CD2 TSC22D2...
2096,TGCTCAACAAACCTAT,1,WDPCP ACBD6 STK4 MAPK14 EZR PIP4K2A EPS15 OSBP...
8939,GACCTCAAGCTGTAAC,16,VCAN LRMDA LYZ IVNS1ABP SLC8A1 NAMPT VIM PTEN ...


In [89]:
import chromadb
import shutil
DB_PATH=Path("../data/testdb")
if DB_PATH.exists():
    shutil.rmtree(DB_PATH)
db_client = chromadb.PersistentClient(path="../data/testdb")
db_client.count_collections()

0

In [91]:
from chromadb.utils import embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=os.getenv("OPENAI_API_KEY"),
    model_name="text-embedding-ada-002")
collection = db_client.create_collection(
    name="cell_embeddings",
    embedding_function=openai_ef) # type:ignore

OperationalError: attempt to write a readonly database